In [ ]:
import numpy as np
import pandas as pd
from mylibs import tools,plots, functions as f

In [ ]:
%matplotlib qt

In [ ]:
exp = tools.load_csv(tools.input_path())
exp.set_info(
    compound=input('Compound: '),
    w=float(input('W mass: ')),
    rho=float(input('Rho: ')),
    
)
plots.temporal_plot(exp,ylabel='Viscosity [cP]')


In [ ]:
test= tools.load_hdf5(tools.input_path())

In [ ]:
test.log

In [ ]:
exp = tools.configurate_data(exp)

In [ ]:
plots.temperature_plot(
    exp,
    title='Viscosity',
    xlabel='Temperature [C]',
    ylabel='Viscosity [cP]',
    interactive=True,

)


In [ ]:
exp_f= (exp
    .apply(f.C_to_K)
    .apply(f.nu_D)
    .apply(f.linearize)
    .group_filter(f.iqr_filter)
    .apply(f.delinearize)
    .apply(f.nu_D)
    .apply(f.K_to_C)
)
print('Filtered')
plots.temperature_plot(
    exp_f,
    title='Viscosity clear',
    xlabel='Temperature [C]',
    ylabel='Viscosity [cP]',
    interactive=True,
    # save=True,
)

In [ ]:
exp_d = (exp_f
    .apply(f.C_to_K)
    .apply(f.nu_D)
    .apply(f.K_to_C)
)
plots.temperature_plot(
    exp_d,
    title='Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
    # save=True
)

In [ ]:
ols_res = tools.create_OLS(exp_d)
exp_ = (exp_d
    .apply(f.C_to_K)
    .apply(f.linearize)
)
ols_res_ = (ols_res
    .apply(f.C_to_K)
    .apply(f.linearize)
    )
plots.comparation_plot(
    exp_d,
    ols_res,
    title='OLS_Linear',
    xlabel='Temperature',
    ylabel='D',
    interactive=True,
    # save=True,
)

In [ ]:
plots.comparation_plot(
    exp,
    ols_res,
    title='OLS_Diffusion',
    xlabel='Temperature [C]',
    ylabel='D [m2/s]',
    interactive=True,
    # save=True,
)

In [ ]:
exp.save_hdf5()